In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/CSV_FILE/final_census.csv")
data.head()

,Unnamed: 0,age,workclass,fnlwgt,education,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,0,90,Private,77053,HS-grad,Widowed,Prof-specialty,Not-in-family,White,Female,0,4356,40,United-States,0
1,1,82,Private,132870,HS-grad,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,0
2,2,66,Private,186061,Some-college,Widowed,Prof-specialty,Unmarried,Black,Female,0,4356,40,United-States,0
3,3,54,Private,140359,Elementary_Middle_School,Seperated,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,0
4,4,41,Private,264663,Some-college,Seperated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,0


In [5]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)
data.head(3)

,age,workclass,fnlwgt,education,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,Private,77053,HS-grad,Widowed,Prof-specialty,Not-in-family,White,Female,0,4356,40,United-States,0
1,82,Private,132870,HS-grad,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,0
2,66,Private,186061,Some-college,Widowed,Prof-specialty,Unmarried,Black,Female,0,4356,40,United-States,0


In [6]:
data.shape

(32536, 14)

In [7]:
data = data.sample(n=20000,ignore_index=True)

In [8]:
X = data.drop('income',axis=1)
y = data['income']

In [9]:
category_variable = [col for col in X.columns if X[col].dtypes == object]
numeric_variable = [col for col in X.columns if X[col].dtypes != object]

In [10]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()

In [11]:
X[category_variable] = X[category_variable].apply(LabelEncoder().fit_transform)

In [12]:
X.head()

,age,workclass,fnlwgt,education,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,66,0,261062,5,3,9,1,4,0,0,0,40,1
1,41,2,126622,4,2,5,4,4,0,0,0,40,1
2,39,2,284166,1,0,9,0,4,1,0,1902,50,1
3,50,2,44368,8,0,3,0,4,1,0,0,40,1
4,84,3,172907,8,0,11,0,4,1,0,0,35,1


In [13]:
y

0        0
1        0
2        1
3        0
4        1
        ..
19995    0
19996    1
19997    0
19998    0
19999    0
Name: income, Length: 20000, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split,GridSearchCV

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=10)

### Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

In [17]:
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [18]:
model.score(X_train,y_train)

1.0

In [19]:
y_predict=model.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score

In [21]:
accuracy_score(y_test,y_predict)

0.7981818181818182

In [22]:
grid_param = {
    'criterion':['gini','entropy'],
    'max_depth': range(2,32,1),
    'min_samples_leaf': range(1,10,1),
    'min_samples_split': range(2,10,1),
    'splitter':['best','random']
}

In [23]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=model,param_grid=grid_param,cv=3)

In [24]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 32),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']})

In [25]:
grid_search.best_params_

{'criterion': 'entropy',
 'max_depth': 7,
 'min_samples_leaf': 9,
 'min_samples_split': 6,
 'splitter': 'best'}

In [ ]:
(criterion='gini',max_depth=7,min_samples_leaf=7,min_samples_split=4,splitter='best')

In [26]:
model_bestparam = DecisionTreeClassifier(criterion='entropy',max_depth=7,min_samples_leaf=9,min_samples_split=6,splitter='best')

In [27]:
model_bestparam.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_leaf=9,
                       min_samples_split=6)

In [28]:
y_predict2 = model_bestparam.predict(X_test)

In [29]:
accuracy_score(y_test,y_predict2)

0.8516666666666667

### Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier
Rf_model = RandomForestClassifier()

In [31]:
Rf_model.fit(X_train,y_train)

RandomForestClassifier()

In [32]:
y_pred_rf = Rf_model.predict(X_test)

In [33]:
accuracy_score(y_test,y_pred_rf)

0.8510606060606061

In [34]:
grid_param = {
    'n_estimators':[90,100],
    'criterion':['gini','entropy'],
    'max_depth': range(2,12,1),
    'min_samples_leaf': range(1,5,1),
    'min_samples_split': range(2,10,1),
    'max_features': ['auto','log2']
}

In [35]:
from sklearn.model_selection import GridSearchCV
grid_search_bestparam = GridSearchCV(estimator=Rf_model,param_grid=grid_param,cv=3)

In [36]:
grid_search_bestparam.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 12),
                         'max_features': ['auto', 'log2'],
                         'min_samples_leaf': range(1, 5),
                         'min_samples_split': range(2, 10),
                         'n_estimators': [90, 100]})

In [37]:
grid_search_bestparam.best_params_

{'criterion': 'gini',
 'max_depth': 11,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [ ]:
(criterion='gini',max_depth=11,max_features='auto',min_samples_leaf=1,min_samples_split=2,n_estimators=100)

In [38]:
Rf_model_bestparam = RandomForestClassifier(criterion='gini',max_depth=11,max_features='auto',min_samples_leaf=1,min_samples_split=2,n_estimators=100)

In [39]:
Rf_model_bestparam.fit(X_train,y_train)

RandomForestClassifier(max_depth=11)

In [40]:
y_pred_rf_bp = Rf_model_bestparam.predict(X_test)

In [41]:
accuracy_score(y_test,y_pred_rf_bp)

0.86

### Bagging Classifier

In [42]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification

model_bagging_svc = BaggingClassifier(base_estimator=SVC(),n_estimators=50, random_state=0)

In [43]:
model_bagging_svc.fit(X_train,y_train)

BaggingClassifier(base_estimator=SVC(), n_estimators=50, random_state=0)

In [44]:
y_pred_bagging = model_bagging_svc.predict(X_test)

In [45]:
accuracy_score(y_test,y_pred_bagging)

0.7875757575757576

### Voting Classifier

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')


In [47]:
eclf1.fit(X_train,y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('rf',
                              RandomForestClassifier(n_estimators=50,
                                                     random_state=1)),
                             ('gnb', GaussianNB())])

In [48]:
y_pred_voting=eclf1.predict(X_test)

In [49]:
accuracy_score(y_test,y_pred_voting)

0.8065151515151515

### Extra Tree Classifier

In [50]:
from sklearn.ensemble import ExtraTreesClassifier
Et_model = ExtraTreesClassifier()

In [51]:
Et_model.fit(X_train,y_train)

ExtraTreesClassifier()

In [52]:
y_pred_et = Et_model.predict(X_test)

In [53]:
accuracy_score(y_test,y_pred_et)

0.8406060606060606

In [54]:
grid_param = {
    'n_estimators':[90,100],
    'criterion':['gini','entropy'],
    'max_depth': range(2,12,1),
    'min_samples_leaf': range(1,10,1),
    'min_samples_split': range(2,10,1),
    'max_features': ['sqrt', 'log2']
}

In [55]:
from sklearn.model_selection import GridSearchCV
grid_search_bestparam_et = GridSearchCV(estimator=Et_model,param_grid=grid_param,cv=3)

In [56]:
grid_search_bestparam_et.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 12),
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'n_estimators': [90, 100]})

In [57]:
grid_search_bestparam_et.best_params_

{'criterion': 'gini',
 'max_depth': 11,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 8,
 'n_estimators': 90}

In [58]:
Et_model_best_param = ExtraTreesClassifier(criterion='gini',max_depth=11,max_features='sqrt',min_samples_leaf= 1,min_samples_split=8,n_estimators=90)

In [60]:
Et_model_best_param.fit(X_train,y_train)

ExtraTreesClassifier(max_depth=11, max_features='sqrt', min_samples_split=8,
                     n_estimators=90)

In [61]:
y_pred_et_bp = Et_model_best_param.predict(X_test)

In [62]:
accuracy_score(y_test,y_pred_et_bp)

0.8412121212121212